In [ ]:
# Choose weights (sum to 1.0)
weights = {"BTC-USD": 0.6, "ETH-USD": 0.4}

# Pivot to aligned returns
pivot = df_ret.pivot(index="date", columns="symbol", values="simple_return").dropna()
common_assets = [a for a in weights if a in pivot.columns]
W = np.array([weights[a] for a in common_assets])

# Portfolio daily returns
port_ret = (pivot[common_assets] @ W)

# Metrics
port_vol_ann = port_ret.std() * np.sqrt(365)
port_sr = sharpe_ratio(port_ret)
port_sor = sortino_ratio(port_ret)
port_var95 = historical_var(port_ret, 0.95)
port_cvar95 = historical_cvar(port_ret, 0.95)
port_curve = (1 + port_ret).cumprod()
port_mdd = max_drawdown(port_curve)

print("=== Portfolio (BTC+ETH) Risk Summary ===")
print(f"Annualized Volatility: {port_vol_ann:0.4f}")
print(f"Sharpe Ratio:         {port_sr:0.4f}")
print(f"Sortino Ratio:        {port_sor:0.4f}")
print(f"VaR(95%):             {port_var95:0.4%}  (one-day)")
print(f"CVaR(95%):            {port_cvar95:0.4%}  (one-day)")
print(f"Max Drawdown:         {port_mdd:0.2%}")

# Plot cumulative returns
plt.figure(figsize=(10,4))
port_curve.plot()
plt.title("BTC+ETH Portfolio: Cumulative Return")
plt.xlabel("Date")
plt.ylabel("Cumulative Return (gross)")
plt.show()
6) Conditional volatility model (GARCH) for risk forecasting
python
Copy
Edit
# Fit a GARCH(1,1) on BTC log-returns
btc_logret = df_ret[df_ret.symbol == "BTC-USD"].set_index("date")["log_return"].dropna()

# arch expects returns in percent or raw; raw is fine
am = arch_model(btc_logret * 100, vol="Garch", p=1, q=1, dist="normal", mean="Constant")
res = am.fit(disp="off")
print(res.summary())

# Forecast 7-day ahead volatility (annualized)
fore = res.forecast(horizon=7)
# sigma2 is variance in percent^2; take sqrt for stdev, then scale to annual
daily_sigma_pct = np.sqrt(fore.variance.values[-1, :])  # percent
daily_sigma = daily_sigma_pct / 100.0
annualized_sigma = daily_sigma * np.sqrt(365)
print("\n7-day ahead annualized volatility forecast:")
for i, v in enumerate(annualized_sigma, start=1):
    print(f"D+{i}: {v:.4f}")